In [1]:
import glob
import pandas as pd
import numpy as np
from data_generator.batch_generator import BatchGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint#, TensorBoard
import pickle
from models import AlexNet, LeNet
from keras import backend as K
from math import sqrt
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [2]:
activations = ['relu', 'lrelu']
#img_treats=['image-treat-1', 'image-treat-2', 'image-treat-3']
img_treats=['abordagem-4', 'abordagem-5', 'abordagem6', 'abordagem7']
#nets = ['lenet', 'alexnet']
nets = ['lenet']

activation, img_treat, net = activations[0], img_treats[1], nets[0]
activation, img_treat, net

('relu', 'abordagem-5', 'lenet')

In [3]:
test_dataset = BatchGenerator(box_output_format=['class_id'])
test_dataset.parse_csv(labels_filename='/home/nicoli/github/alexnet/dataset/csv/imdb_csv/imdb_age_regression_test_split_47950-70-10-20.csv', 
                        images_dir='/home/nicoli/github/alexnet/dataset/imdb-hand-crop/',
                        input_format=['image_name', 'class_id'])

In [4]:
img_height, img_width, img_depth = (224,224,3)

epochs = 90
melhor_epoca='08'
train_batch_size = 128
shuffle = True
ssd_train = False

validation_batch_size = 32


# In[15]:

test_generator = test_dataset.generate(batch_size=train_batch_size,
                                       shuffle=shuffle,
                                       ssd_train=ssd_train,
                                       divide_by_stddev = 225,
                                       #equalize=True,
                                       returns={'processed_labels'},
                                       resize=(img_height, img_width))

In [5]:
print("Number of images in the dataset:", test_dataset.get_n_samples())

Number of images in the dataset: 9590


In [6]:
steps = test_dataset.get_n_samples()/train_batch_size

In [7]:
if net == 'alexnet':
    net_obj = AlexNet
elif net == 'lenet':
    net_obj = LeNet
weights_path = glob.glob('callbacks/' + net + '/age/consolidados/class-weights-' + img_treat + '-' + activation + '*')[0]
#weights_path = glob.glob('callbacks/' + net + '/age/class-weights-' + img_treat + '-' + activation + '.' + melhor_epoca + '*')[0]
model = net_obj(1, img_width, img_height, img_depth, activation, weights_path=weights_path)


# In[7]:


model.summary()
#alexnet.model.load_weights('callbacks/alexnet/age/weights.24-1658.03.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 220, 220, 6)       456       
_________________________________________________________________
activation_1 (Activation)    (None, 220, 220, 6)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 6)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 106, 106, 16)      2416      
_________________________________________________________________
activation_2 (Activation)    (None, 106, 106, 16)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 53, 53, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 44944)             0         
__________

/home/nicoli/github/alexnet/models.py:109: UserWarning: Considering a regression task with output function being relu
  warnings.warn(warning)


In [8]:
optimizer = Adam(lr=0.0000001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0000000001, amsgrad=True)

In [9]:
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])

In [10]:
stats = model.evaluate_generator(test_generator,steps=30)


sqrt(stats[0]), stats

(13.030968210663557, [169.8061325073242, 10.089216232299805])

In [11]:
start = weights_path.find(activation)+len(activation)+1
end = start + 2
epoca = int(weights_path[start:end]) + 1

In [12]:
try:
    df_results = pd.read_csv('graficos/results_teste.csv', index_col=0)

except FileNotFoundError:
    df_results = pd.DataFrame({'Rede':[], 'Ativação':[], 'RMSE':[], 'MAE':[], 'FASE': [], 'Epoca': []})
df_results = df_results.append({'Rede':net, 'Ativação':activation, 'RMSE': sqrt(stats[0]), 'MAE':stats[1], 'FASE': img_treat, 'Epoca': epoca}, ignore_index=True)

# In[13]:

#df_results.to_csv('graficos/results_teste_' + img_treat + '.csv', index_col=0)
df_results.to_csv('graficos/results_teste.csv')



In [18]:
df_results

,Ativação,FASE,MAE,RMSE,Rede,Epoca
0,relu,image-treat-1,10.537234,13.553820,lenet,4
1,lrelu,image-treat-1,38.330450,40.826534,lenet,8
2,relu,image-treat-1,11.032492,13.764649,alexnet,5
3,lrelu,image-treat-1,39.278125,41.974881,alexnet,5
4,relu,image-treat-2,37.854424,40.272234,lenet,39
5,lrelu,image-treat-2,38.501557,41.065953,lenet,21
6,relu,image-treat-2,11.591327,14.595028,alexnet,16
7,lrelu,image-treat-2,28.062363,31.811022,alexnet,16
8,relu,image-treat-3,38.656497,41.200746,lenet,46
9,lrelu,image-treat-3,38.261533,40.855255,lenet,38


In [14]:
prev = next(test_generator)


#prev[1]
predict = model.predict(prev[0])


In [15]:
print(predict[:5])

[[53.75101 ]
 [36.87372 ]
 [36.668808]
 [44.03962 ]
 [55.682224]]
